In [1]:
from beta import Beta

In [5]:
beta = Beta('PYPL', start='2019-1-1', end='2020-8-1')

PYPL/SPY::
Correlation: 0.5734573548346732
Covariance: 285.1883456625234
Variance: 477.74349159741473

BETA:  0.5969486778541948


### Porfolio Creation

In [126]:
test_port = ['SQ', 'PYPL', 'AAPL', 
             'NVDA', 'CRSP', 'AMD', 'T', 
             'SLV', 'GLD',]

### Process 
  1. Retrieve the daily charts for each ticker
  2. Calculate Sharpe ratio relative to S&P500 Xyear
  3. Minimize for the -Sharpe ratio, such that sum of all allocations = 1.0

In [84]:
import matplotlib.pyplot as plt
import yfinance as yf
import pandas as pd
import numpy as np
import math
import sys

In [128]:
# for each ticker in portfolio 
# retrieve data from the last x time 
# store data into a 3d dataframe where
# x = ticker, y = day, z = O,H,L,C,V
class Portfolio():
    benchmark = None
    portfolio = []
    
    stdev = pd.DataFrame()
    
    daily_open = pd.DataFrame()
    daily_high = pd.DataFrame()
    daily_low = pd.DataFrame()
    daily_close = pd.DataFrame()
    daily_volume = pd.DataFrame()
    
    def __init__(self, portfolio=None, benchmark='SPY', start='2019-3-16', end='2020-8-1'): #todo, default end today
        self.start = start
        self.end = end
        self.benchmark = benchmark
        data = self.pull_market_data(self.benchmark)
        self.initialize_charts(data)
        if portfolio != None:
            print('Portfolio:', portfolio)
            self.add_securities(portfolio)
        #print(self.daily_close, ",/,", self.daily_close.std(axis=0))
        self.stdev = self.daily_close.std(axis=0)
        
    def pull_market_data(self, ticker):
        df = pd.DataFrame()
#         try:
        #print(type(ticker), ticker)
        md = yf.Ticker(ticker)
        df = md.history(period='max', start=self.start, end=self.end)
        return df
#         except Exception as e:
#             print(str(e))
        
        #return -1
    
    def initialize_charts(self, df):
        ''' Takes the benchmark df and initializes a dataframe for each element 
            of a daily candle: Open, High, Low, Close, and Volume'''
        self.daily_open = df['Open'].rename(columns={"Open": self.benchmark}).rename(self.benchmark)
        self.daily_high = df['High'].rename(columns={"High": self.benchmark}).rename(self.benchmark)
        self.daily_low = df['Low'].rename(columns={"Low": self.benchmark}).rename(self.benchmark)
        self.daily_close = df['Close'].rename(columns={"Close": self.benchmark}).rename(self.benchmark)
        self.daily_volume = df['Volume'].rename(columns={"Volume": self.benchmark}).rename(self.benchmark)
        return 0
    
    def initialize_indicators(self, df):
        ''' Initializes a dataframe for each indicator based on the data in the 
            daily candles for the benchmark'''
        return 0
        
    def add_securities(self, securities):
        ''' Takes a list, securities, and adds each element in the list to our portfolio'''
        securities = [ s for s in securities if s not in self.portfolio]
        securities = [ s for s in securities if self.validate_ticker(s)]
        self.portfolio.extend( securities )
        
        for asset in self.portfolio:
            df = self.pull_market_data(asset)
            #TODO: modularize code below with self.initialize_charts()
            tmp_open = df['Open'].rename(columns={"Open": asset}).rename(asset)
            tmp_high = df['High'].rename(columns={"High": asset}).rename(asset)
            tmp_low = df['Low'].rename(columns={"Low": asset}).rename(asset)
            tmp_close = df['Close'].rename(columns={"Close": asset}).rename(asset)
            tmp_volume = df['Volume'].rename(columns={"Volume": asset}).rename(asset)
            
            #print(self.daily_open.name, self.daily_open[:1], '\n\n', tmp_open.name,tmp_open[:1])
            self.daily_open = pd.merge(self.daily_open, tmp_open, on='Date', how='left').bfill()
            self.daily_high = pd.merge(self.daily_high, tmp_high, on='Date', how='left').bfill()
            self.daily_low = pd.merge(self.daily_low, tmp_low, on='Date', how='left').bfill()
            self.daily_close = pd.merge(self.daily_close, tmp_close, on='Date', how='left').bfill()
            self.daily_volume = pd.merge(self.daily_volume, tmp_volume, on='Date', how='left').bfill()
            
            
        
    def remove_securities(self, securities):
        ''' Takes a list, securities, and removes each element from our portfolio'''
        self.portfolio = [ s for s in self.portfolio if s not in securities]
    
    # TODO: verify that ticker legit 
    def validate_ticker(self, ticker):
        return True

In [129]:
p = Portfolio(portfolio=test_port)
#p.stdev
p.daily_close

Portfolio: ['SQ', 'PYPL', 'AAPL', 'NVDA', 'CRSP', 'AMD', 'T', 'SLV', 'GLD']


,SPY,SQ,PYPL,AAPL,NVDA,CRSP,AMD,T,SLV,GLD
Date,,,,,,,,,,
2019-03-18,275.46,75.23,100.27,185.09,168.29,37.88,23.25,28.08,14.39,123.04
2019-03-19,275.53,76.81,101.54,183.62,175.02,37.24,26.00,27.97,14.40,123.38
2019-03-20,274.70,75.26,102.31,185.23,173.72,37.10,25.70,27.81,14.54,124.18
2019-03-21,277.81,78.53,104.15,192.05,183.22,37.86,27.89,28.32,14.50,123.68
2019-03-22,272.46,75.09,101.27,188.07,176.81,35.59,26.37,28.32,14.45,123.97
...,...,...,...,...,...,...,...,...,...,...
2020-07-27,323.22,125.75,177.98,379.24,416.86,92.12,68.97,29.29,22.83,182.23
2020-07-28,321.17,123.51,176.27,373.01,408.62,86.40,67.61,29.69,22.77,183.75
2020-07-29,325.12,128.55,184.60,380.16,418.62,85.53,76.09,29.56,22.57,185.13


In [130]:
exp_df = p.daily_close.iloc[-1]/p.daily_close.iloc[0]-1 # expected return

# get sharpe ratio for each expected return
assets = []
contributions = []
volatilities = []
for k,v in exp_df.iteritems(): 
    #print('k = sqrt(252) =', math.sqrt(252))
    K = math.sqrt(252)
    asset = {'returns': 0.0, 'numerator': 0.0, 
           'denominator': 0.0, 'sharpe': 0.0, 
           'alloc': 0.0}
    sharpe = {}
    if k != 'SPY': # if we aren't the benchmark
        asset['returns'] = exp_df[k]
        asset['numerator'] = asset['returns']-exp_df['SPY']
        asset['denominator'] = p.stdev[k]
        asset['sharpe'] = K*(asset['numerator']/asset['denominator'])
        asset['alloc'] = 1.0/exp_df.shape[0]
        print(f'{k}::\n',
              f'Returns: {asset["returns"]}\n',
              f'Volatility: {asset["denominator"]}\n', 
              f'Sharpe: {asset["sharpe"]}\n',
              f'Allocation: {asset["alloc"]}\n')
        contributions.append(asset["returns"]*asset["alloc"])
        volatilities.append(asset["denominator"]*asset["alloc"])
        #print('Sharpe Ratio for', k,'=', sharpe[k])
        
# calculate sharpe of portfolio as a whole, given some array of default allocations and objective
print('\nTOTAL PORTFOLIO:::\n', 
      f'Returns: {np.mean(contributions)}\n', 
      f'Volatility: {np.mean(volatilities)}\n', 
      f'Sharpe: {(np.mean(contributions)-exp_df["SPY"])/np.mean(volatilities)}\n')

SQ::
 Returns: 0.7260401435597499
 Volatility: 17.162499035929166
 Sharpe: 0.5001013460435015
 Allocation: 0.1

PYPL::
 Returns: 0.955420365014461
 Volatility: 22.548342716612062
 Sharpe: 0.5421368280964167
 Allocation: 0.1

AAPL::
 Returns: 1.2963963477227294
 Volatility: 59.5679175705631
 Sharpe: 0.2960840942286428
 Allocation: 0.1

NVDA::
 Returns: 1.5229663081585358
 Volatility: 76.66995540220324
 Sharpe: 0.2769507224414241
 Allocation: 0.1

CRSP::
 Returns: 1.2560718057022173
 Volatility: 13.940115216596281
 Sharpe: 1.2192855220606735
 Allocation: 0.1

AMD::
 Returns: 2.3303225806451615
 Volatility: 11.3878028121889
 Sharpe: 2.9900573094146576
 Allocation: 0.1

T::
 Returns: 0.05341880341880345
 Volatility: 3.2997578388960487
 Sharpe: -0.6347568482570388
 Allocation: 0.1

SLV::
 Returns: 0.5740097289784571
 Volatility: 1.6768829211110665
 Sharpe: 3.679195953078078
 Allocation: 0.1

GLD::
 Returns: 0.5070708712613783
 Volatility: 15.008027102659454
 Sharpe: 0.3402818637524883
 Allo